In [ ]:
import os
import requests
import json
import time
from IPython.display import Video, display # Jupyter Notebook 출력을 위해 Video, display 모듈 추가
from datetime import datetime # 현재 시간 처리를 위해 datetime 모듈 추가

from dotenv import load_dotenv

# .env 환경변수 로드 (API 키와 엔드포인트를 환경 변수에서 가져올 경우 필요)
load_dotenv() 

# --- 환경 변수 설정 (사용자의 기존 설정 유지) ---
# 이러한 환경 변수를 설정하거나 다음 값을 편집해야 합니다.
# 실제 사용 시에는 환경 변수에서 안전하게 가져오는 것이 좋습니다.
endpoint = os.getenv("ENDPOINT_URL", "https://8a000-openai.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "sora")
subscription_key = os.getenv("AZURE_CHRIS_KEY") # API 키는 환경 변수에서 가져오는 것을 권장
if not subscription_key:
    # 환경 변수가 설정되지 않은 경우, 여기에 실제 API 키를 넣어야 합니다.
    # 예시: subscription_key = "YOUR_API_KEY_HERE"
    print("경고: SUBSCRIPTION_KEY가 설정되지 않았습니다. 실제 사용 시 이 값을 채워주세요.")

api_version = "preview"
path = f'openai/v1/video/generations/jobs'
params = f'?api-version={api_version}'
constructed_url = endpoint + path + params

headers = {
  'Api-Key': subscription_key,
  'Content-Type': 'application/json',
}

body = {
  "prompt": "선생님한테 배고프다고 소리치며 교실을 나가버리는 학생",
  "n_variants": "1",
  "n_seconds": "5",
  "height": "480",
  "width": "854",
  "model": deployment,
}

print("--- 1. 비디오 생성 작업 요청 ---")
job_response = requests.post(constructed_url, headers=headers, json=body)

if not job_response.ok:
    print("❌ 비디오 생성 요청에 실패했습니다.")
    print(json.dumps(job_response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
else:
    job_response_json = job_response.json()
    print(json.dumps(job_response_json, sort_keys=True, indent=4, separators=(',', ': ')))
    
    job_id = job_response_json.get("id")
    status = job_response_json.get("status")
    status_url = f"{endpoint}openai/v1/video/generations/jobs/{job_id}?api-version={api_version}"

    print(f"\n⏳ 2. 작업 상태 확인 (Job ID: {job_id})")
    # 비디오 생성이 완료(succeeded)되거나 실패(failed)할 때까지 상태를 반복 확인합니다.
    while status not in ["succeeded", "failed"]:
        time.sleep(5) # 5초 대기
        try:
            job_response = requests.get(status_url, headers=headers, timeout=10).json()
            status = job_response.get("status")
            print(f"현재 상태: {status}")
        except requests.exceptions.RequestException as e:
            print(f"상태 확인 중 오류 발생: {e}")
            status = "failed" # 통신 오류 시 실패로 간주하고 루프 종료

    if status == "succeeded":
        print(f"\n✅ 3. 비디오 생성 성공. 파일 다운로드 시작.")
        generations = job_response.get("generations", [])
        if generations:
            generation_id = generations[0].get("id")
            video_url = f'{endpoint}openai/v1/video/generations/{generation_id}/content/video{params}'
            
            video_response = requests.get(video_url, headers=headers, stream=True)
            if video_response.ok:
                now = datetime.now()
                timestamp_str = now.strftime("%y%m%d%H%M") 
            
                output_filename = f"files/{timestamp_str}_output_video.mp4"
                
                # 'files' 디렉토리가 없으면 생성
                os.makedirs(os.path.dirname(output_filename), exist_ok=True)
                
                # 파일 저장
                with open(output_filename, "wb") as file:
                    for chunk in video_response.iter_content(chunk_size=8192):
                        file.write(chunk)
                print(f'   -> 생성된 비디오가 "{output_filename}"으로 저장되었습니다.')
                
                # 4. Jupyter Notebook에 비디오 직접 출력
                print("\n🖥️ 4. Jupyter Notebook에 비디오 출력:")
                try:
                    display(Video(output_filename))
                except NameError:
                    print("   -> 경고: IPython.display.Video를 사용할 수 없습니다. Jupyter Notebook 환경에서 실행 중인지 확인하세요.")
                
            else:
                print(f"❌ 비디오 콘텐츠 다운로드에 실패했습니다. 상태 코드: {video_response.status_code}")
        else:
            print("⚠️ Status는 succeeded이지만, 반환된 비디오 결과물이 없습니다.")
            
    elif status == "failed":
        print("\n❌ 3. 비디오 생성 작업에 실패했습니다.")
        print(json.dumps(job_response, sort_keys=True, indent=4, separators=(',', ': ')))

--- 1. 비디오 생성 작업 요청 ---
{
    "created_at": 1764214819,
    "expires_at": null,
    "failure_reason": null,
    "finished_at": null,
    "generations": [],
    "height": 480,
    "id": "task_01kb1pbjn2er5aq684vhse4k3c",
    "inpaint_items": null,
    "model": "sora",
    "n_seconds": 5,
    "n_variants": 1,
    "object": "video.generation.job",
    "prompt": "\uc120\uc0dd\ub2d8\ud55c\ud14c \ubc30\uace0\ud504\ub2e4\uace0 \uc18c\ub9ac\uce58\uba70 \uad50\uc2e4\uc744 \ub098\uac00\ubc84\ub9ac\ub294 \ud559\uc0dd",
    "status": "preprocessing",
    "width": 854
}

⏳ 2. 작업 상태 확인 (Job ID: task_01kb1pbjn2er5aq684vhse4k3c)
현재 상태: preprocessing
현재 상태: running
현재 상태: processing
현재 상태: succeeded

✅ 3. 비디오 생성 성공. 파일 다운로드 시작.
   -> 생성된 비디오가 "files/2511271240_output_video.mp4"으로 저장되었습니다.

🖥️ 4. Jupyter Notebook에 비디오 출력:
